In [1]:
import gym
import random
import copy
import numpy as np
from collections import deque
from base64 import b64encode

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader, IterableDataset

from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

from gym.wrappers import RecordVideo, RecordEpisodeStatistics, TimeLimit


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
num_gpus = torch.cuda.device_count()

D:\multi_agents_rl\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class DQN(nn.Module):
    def __init__(self, hidden_size, obs_size, n_actions):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)     
        )
    
    def forward(self, x):
        return self.net(x.float())


In [3]:
def epsilon_greedy(state, env, net, epsilon=0.0):
    if np.random.random() < epsilon:
        action = env.action_space.sample()
    else:
        state = torch.tensor([state]).to(device)
        q_values = net(state)
        _, action = torch.max(q_values, dim=1)
        action = int(action.item())
    return action

In [4]:
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
        
    def __len__(self):
        return len(self.buffer)
    
    def append(self, experience):
        self.buffer.append(experience)
        
    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

In [5]:
class RLDataset(IterableDataset):
    def __init__(self, buffer, sample_size=400):
        self.buffer = buffer
        self.sample_size = sample_size
    
    def __iter__(self):
        for experience in self.buffer.sample(self.sample_size):
            yield experience

In [6]:
def create_environment(name):
    env = gym.make(name, render_mode="rgb_array")
    env = TimeLimit(env, max_episode_steps=400)
    env = RecordVideo(env, video_folder='./videos', episode_trigger=lambda x: x % 50 == 0)
    env = RecordEpisodeStatistics(env)
    return env

In [7]:
class DeepQLearning(LightningModule):
    def __init__(self, env_name, policy=epsilon_greedy, capacity=100_000, 
               batch_size=256, lr=1e-3, hidden_size=128, gamma=0.99, 
               loss_fn=F.smooth_l1_loss, optim=AdamW, eps_start=1.0, eps_end=0.15, 
               eps_last_episode=100, samples_per_epoch=1_000, sync_rate=10):
    
        super().__init__()
        self.env = create_environment(env_name)

        obs_size = self.env.observation_space.shape[0]
        n_actions = self.env.action_space.n

        self.q_net = DQN(hidden_size, obs_size, n_actions)

        self.target_q_net = copy.deepcopy(self.q_net)

        self.policy = policy
        self.buffer = ReplayBuffer(capacity=capacity)
        self.save_hyperparameters()

        while len(self.buffer) < self.hparams.samples_per_epoch:
            print(f"{len(self.buffer)} samples in experience buffer. Filling...")
            self.play_episode(epsilon=self.hparams.eps_start)
            
    @torch.no_grad()
    def play_episode(self, policy=None, epsilon=0.):
        state = self.env.reset()
        state = state[0]
        done = False
        while not done:
            if policy:
                action = policy(state, self.env, self.q_net, epsilon=epsilon)
            else:
                action = self.env.action_space.sample()
            
            next_state, reward, done, tru , _ = self.env.step(action)
            if tru:
                done = tru
            exp = (state, action, reward, done, next_state)
            self.buffer.append(exp)
            state = next_state
            
        self.env.close()
        
        
    @torch.no_grad()
    def play_test_episode(self):
        self.q_net.to(device)
        state = self.env.reset()
        state = state[0]
        done = False
        while not done:
            action = self.policy(state, self.env, self.q_net, epsilon=0)
            next_state, reward, done, tru , _ = self.env.step(action)
            if tru:
                done = tru
            state = next_state
        self.env.close()

    def forward(self, x):
        return self.q_net(x)

    
    def configure_optimizers(self):
        q_net_optimizer = self.hparams.optim(self.q_net.parameters(), lr=self.hparams.lr)
        return [q_net_optimizer]

     # Create dataloader.
    def train_dataloader(self):
        dataset = RLDataset(self.buffer, self.hparams.samples_per_epoch)
        dataloader = DataLoader(
            dataset=dataset,
            batch_size=self.hparams.batch_size
        )
        return dataloader
    
    def training_step(self, batch, batch_idx):
        states, actions, rewards, dones, next_states = batch
        actions = actions.unsqueeze(1)
        rewards = rewards.unsqueeze(1)
        dones = dones.unsqueeze(1)
        
        state_action_values = self.q_net(states).gather(1, actions)

        next_action_values, _ = self.target_q_net(next_states).max(dim=1, keepdim=True)
        next_action_values[dones] = 0.0

        expected_state_action_values = rewards + self.hparams.gamma * next_action_values

        loss = self.hparams.loss_fn(state_action_values, expected_state_action_values)
        self.log('episode/Q-Error', loss)
        return loss
    
    # Training epoch end.
    def training_epoch_end(self, training_step_outputs):
        epsilon = max(
            self.hparams.eps_end,
            self.hparams.eps_start - self.current_epoch / self.hparams.eps_last_episode
        )

        self.play_episode(policy=self.policy, epsilon=epsilon)
        self.log('episode/Return', self.env.return_queue[-1])

        if self.current_epoch % self.hparams.sync_rate == 0:
            self.target_q_net.load_state_dict(self.q_net.state_dict())
            
    def save_model(self):
        torch.save(self.q_net.state_dict(), "./model")
        
    def load_model(self):
        self.q_net.load_state_dict(torch.load( "./model"))


In [8]:
algo = DeepQLearning('LunarLander-v2')

checkpoint_callback = ModelCheckpoint(dirpath="./checkpoints", save_top_k=1,mode="max", monitor="episode/Return")

trainer = Trainer(
   accelerator='gpu',
     devices=num_gpus,
    max_epochs=10_000,
    callbacks=[EarlyStopping(monitor='episode/Return', mode='max', patience=1000), checkpoint_callback]
)

trainer.fit(algo)


D:\multi_agents_rl\env\lib\site-packages\gym\wrappers\record_video.py:75: UserWarning: WARN: Overwriting existing videos at D:\multi_agents_rl\videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


0 samples in experience buffer. Filling...
Moviepy - Building video D:\multi_agents_rl\videos\rl-video-episode-0.mp4.
Moviepy - Writing video D:\multi_agents_rl\videos\rl-video-episode-0.mp4



GPU available: True (cuda), used: True                                                                                                                                         
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
D:\multi_agents_rl\env\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:616: UserWarning: Checkpoint directory D:\multi_agents_rl\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Moviepy - Done !
Moviepy - video ready D:\multi_agents_rl\videos\rl-video-episode-0.mp4
91 samples in experience buffer. Filling...
198 samples in experience buffer. Filling...
275 samples in experience buffer. Filling...
366 samples in experience buffer. Filling...
465 samples in experience buffer. Filling...
568 samples in experience buffer. Filling...
664 samples in experience buffer. Filling...
722 samples in experience buffer. Filling...
840 samples in experience buffer. Filling...
906 samples in experience buffer. Filling...


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type | Params
--------------------------------------
0 | q_net        | DQN  | 18.2 K
1 | target_q_net | DQN  | 18.2 K
--------------------------------------
36.4 K    Trainable params
0         Non-trainable params
36.4 K    Total params
0.145     Total estimated model params size (MB)
D:\multi_agents_rl\env\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...


t:   0%|                                                                                                                                       | 0/1 [00:00<?, ?it/s, now=None]
                                                                                                                                                                               

Moviepy - Done !                                          
Moviepy - video ready D:\multi_agents_rl\videos\rl-video-episode-1700.mp4
Moviepy - Building video D:\multi_agents_rl\videos\rl-video-episode-1700.mp4.
Moviepy - Writing video D:\multi_agents_rl\videos\rl-video-episode-1700.mp4

Epoch 1689: : 4it [00:01,  2.23it/s, loss=0.597, v_num=11]


t:  93%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊         | 371/401 [00:00<00:00, 663.92it/s, now=None]
                                                                                                                                                                               

Moviepy - Done !                                          
Moviepy - video ready D:\multi_agents_rl\videos\rl-video-episode-1700.mp4
Epoch 1717: : 4it [00:00, 90.94it/s, loss=0.404, v_num=11] 

D:\multi_agents_rl\env\lib\site-packages\pytorch_lightning\trainer\trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [15]:
algo = DeepQLearning.load_from_checkpoint("./checkpoints/epoch=2255-step=9024.ckpt")

0 samples in experience buffer. Filling...
Moviepy - Building video D:\multi_agents_rl\videos\rl-video-episode-0.mp4.
Moviepy - Writing video D:\multi_agents_rl\videos\rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready D:\multi_agents_rl\videos\rl-video-episode-0.mp4
97 samples in experience buffer. Filling...
222 samples in experience buffer. Filling...
314 samples in experience buffer. Filling...
382 samples in experience buffer. Filling...
496 samples in experience buffer. Filling...
565 samples in experience buffer. Filling...
653 samples in experience buffer. Filling...
748 samples in experience buffer. Filling...
866 samples in experience buffer. Filling...
936 samples in experience buffer. Filling...
